## Importing relevant libraries

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

## Helper functions

In [ ]:
def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))

    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]

    return op_new

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

def compute_expectation_x_squared_simpson(x, y, n):
    """
    Computes the expectation value of x^2 using Simpson's rule for numerical integration.
    
    Parameters:
    x (array-like): Discrete values of x.
    y (array-like): Corresponding values of the probability density function (PDF) at x.
    
    Returns:
    float: The expectation value of x^2.
    """
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)
    
    # Compute x^2
    x_squared = x**n
    
    # Check if the number of intervals is even, if not make it even by truncating the last point
    if len(x) % 2 == 0:
        x = x[:-1]
        y = y[:-1]
        x_squared = x_squared[:-1]
    
    # Compute the integral using Simpson's rule
    h = (x[-1] - x[0]) / (len(x) - 1)
    integral = y[0] * x_squared[0] + y[-1] * x_squared[-1] + \
               4 * np.sum(y[1:-1:2] * x_squared[1:-1:2]) + \
               2 * np.sum(y[2:-2:2] * x_squared[2:-2:2])
    integral *= h / 3
    
    return integral

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

def get_fidelity(zeromode_classic, zeromode_vqe):
    # Function to compute the infidelity

    overlap = np.dot(np.transpose(zeromode_vqe), zeromode_classic)
    fidelity = 1 - overlap ** 2
    return fidelity

def expect_value(zeromode, matrix):
    
    value = np.dot(matrix, zeromode)
    expect = np.dot(np.transpose(zeromode), value)
    
    return expect

## VQE implementation and relative error computation

In [ ]:
# Define the VQE computation with varying ansatz depth
def vqe_result(matrix, optimizer, depth, seed):
    # Function to compute the execution time for different optimizers and ansatz depths (averaged over 10 independent VQE runs in each case)

    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Define the qubit Hamiltonian
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)

    # Compute using NumPyMinimumEigensolver for reference
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qub_hamiltonian)
    
    # Define the circuit ansatz with variable depth
    ansatz = RealAmplitudes(num_qubits = num_qubits, reps = depth)
    
    # Initialize the optimizer and the initial point
    np.random.seed(seed)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    
    # Initializing the estimator
    estimator = Estimator()
    
    # Run the VQE algorithm
    @dataclass
    class VQELog:
        values: list
        parameters: list
        
        def update(self, count, parameters, mean, _metadata):
            self.values.append(mean)
            self.parameters.append(parameters)

    log = VQELog([], [])

    vqe = VQE(estimator,
              ansatz, optimizer, initial_point = initial_point, callback = log.update)
    result = vqe.compute_minimum_eigenvalue(qub_hamiltonian)

    # Get the optimizer runtime
    time = result.optimizer_time

    # Estimate the zeromode
    optimal_params = result.optimal_point
    final_circuit = ansatz.assign_parameters(optimal_params)
    zeromode_vqe = Statevector.from_instruction(final_circuit)

    return result.optimal_value, zeromode_vqe, time

def compute_errors(expect_classical, expect_quantum):

    error = np.abs(expect_classical - expect_quantum) / expect_classical
    return error

## Estimating performance metrics at different ansatz depths (example of an implementation)

In [ ]:
## Running computations
# Get the optimizers
optimizers = [SLSQP(maxiter = 5000), P_BFGS(maxfun = 5000), NELDER_MEAD(maxiter = 5000), POWELL(maxiter = 5000)]

# Define the varying ansatz depths
ansatz_depths = [2, 4, 6, 8, 10, 12]

## Run calculations for all depths
# Initialize
depth_results = {depth: {'eigenvalues': [], 'times': [], 'relative_errors': []} for depth in ansatz_depths}
runs = 10

# Loop through each optimizer
for optimizer in optimizers:
    optimizer_name = optimizer.__class__.__name__
    print(f"\nRunning VQE for optimizer: {optimizer_name}")
    
    # Loop through each ansatz depth
    for depth in ansatz_depths:
        print(f"Running for depth = {depth}")
        
        # Temporary storage for the current depth
        eigenvalues_temp = []
        clock_times_temp = []
        relative_errors_temp = []

        # Perform the 10 independent VQE runs
        for i in range(runs):
            # Set a unique seed for each run
            seed = i + 1

            # Run VQE
            eigenvalue, zeromode, time = vqe_result(matrix = matrix, optimizer = optimizer, depth = depth, seed = seed)
            eigenvalues_temp.append(eigenvalue)
            clock_times_temp.append(time)

            # Compute the relative errors in <x^2>
            zeromode = zeromode.data.tolist()
            x, y = get_pdf(nmax, x, dx, L, shift, np.real(zeromode), normalize = True, make_even = True)
            expect_quantum = compute_expectation_x_squared_simpson(x, y, 2)
            error = compute_errors(expect_classical, expect_quantum)
            relative_errors_temp.append(error)

        # Compute averages over the 10 runs
        eigenvalue_average = np.sum(np.abs(eigenvalues_temp)) / runs
        clock_time_average = np.sum(clock_times_temp) / runs
        relative_errors_average = np.sum(relative_errors_temp) / runs
        
        # Store the results for the current depth
        depth_results[depth]['eigenvalues'].append(eigenvalue_average)
        depth_results[depth]['times'].append(clock_time_average)
        depth_results[depth]['relative_errors'].append(relative_errors_average)

        print(f"Depth {depth}: Average eigenvalue = {eigenvalue_average}, Average runtime = {clock_time_average}s, \
        Average relative error = {relative_errors_average}")